# LSSTCam pointing errors
# Looking at the results of homing the mount.

Craig Lage - 18-Apr-25

In [ ]:
import os, sys
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
client = makeEfdClient()
eventMaker = TMAEventMaker()

## Get the MTMount state changes

In [ ]:
startDay = 20250708
endDay = 20250708
dayObs = startDay
enableDict = {}
homeDict = {}
while dayObs <= endDay:
    try:
        start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
        end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
        states = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_summaryState",
            columns=['summaryState'],
            begin=start,
            end=end
        )
        enables = states[states['summaryState'] == 2]
        
        homes = getEfdData(
        client,
        "lsst.sal.MTMount.command_homeBothAxes",
        begin=start,
        end=end
    )
        print(f"There were {len(enables)} enable events and {len(homes)} home events on {dayObs}")
        enableDict[dayObs] = enables
        homeDict[dayObs] = homes
        dayObs = calcNextDay(dayObs)
    except:
        dayObs = calcNextDay(dayObs)
        continue

# Now look at how the offset varies with Az/El

In [ ]:
dayObs = 20250708

testStart = Time("2025-07-08T20:27:00", scale='utc')
testEnd = Time("2025-07-08T21:25:00", scale='utc')
events = eventMaker.getEvents(dayObs)


In [ ]:
dayObs = 20250708

testStart = Time("2025-07-08T20:27:00", scale='utc')
testEnd = Time("2025-07-08T21:25:00", scale='utc')
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Offset_vs_AzEl_{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
#events = eventMaker.getEvents(dayObs)
#myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0) and (e.begin > testStart) and (e.end < testEnd))]
goodEvents = [e for e in events if ((e.type.name=='SLEWING') and (e.duration > 10.0) and (e.begin > testStart) and (e.end < testEnd))]
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria.")

azs = []
els = []
azOffsets = []
elOffsets = []
seqNums = []
for event in goodEvents:
    seqNums.append(event.seqNum)
    if event.seqNum < 5:
        continue
    axs = []
    axs.append(fig.add_subplot(1,2,1))
    axs.append(fig.add_subplot(1,2,2))
    plt.subplots_adjust(wspace = 0.5)
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -2.0,
        postPadding = -2.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -2.0,
        postPadding = -2.0
    )
    encoder = getEfdData(
        client,
        "lsst.sal.MTMount.encoder",
        event=event, 
        prePadding = -2.0,
        postPadding = -2.0
    )

    azValues = np.asarray(az["actualPosition"])
    azs.append(azValues[0])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0      
    
    elValues = np.asarray(el["actualPosition"])
    els.append(elValues[0])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    plt.suptitle(f"{dayObs}, Slewing event {event.seqNum}, {event.begin.isot} -> {event.end.isot}")
    axs[0].scatter(azValues, azEncInterp)
    azFit = np.polyfit(azValues, azEncInterp, 1)
    #print(fit)
    axs[0].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0].transAxes)
    axs[0].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0].transAxes)
    azOffsets.append((azFit[1]*3600))
    axs[0].set_xlabel("Azimuth (degrees)")
    axs[0].set_ylabel("Azimuth Encoder average(degrees)")
    axs[0].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[1].scatter(elValues, elEncInterp)
    elFit = np.polyfit(elValues, elEncInterp, 1)
    #print(fit)
    axs[1].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1].transAxes)
    axs[1].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1].transAxes)
    elOffsets.append((elFit[1]*3600))
    axs[1].set_xlabel("Elevation (degrees)")
    axs[1].set_ylabel("Elevation Encoder average(degrees)")
    pdf.savefig(fig)
    fig.clf()
pdf.close()  

fig = plt.figure(figsize=(8,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Encoder offsets {dayObs}")
plt.subplot(1,2,1)
plt.scatter(azs, azOffsets)
plt.ylim(-5000,5000)
plt.xlabel("Azimuth(degrees)")
plt.ylabel("Azimuth offset (arcseconds)")
plt.subplot(1,2,2)
plt.scatter(els, elOffsets)
plt.ylim(-5000,5000)
plt.xlabel("Elevation(degrees)")
plt.ylabel("Elevation offset (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Offset_Summary_{dayObs}.png")

# Now calculate the delays for all of the tracking events

In [ ]:
from scipy.optimize import minimize
maxDeltaT = 10.0
def calcDeltaT(params, args):
    # This calculates the deltaT needed
    # to minimize the offset
    [values, valTimes, encVals, encTimes, refTimes] = args
    [deltaT] = params
    valsInterp = np.interp(refTimes, valTimes, values)
    encInterp = np.interp(refTimes, encTimes - deltaT, encVals)
    error = np.sum((valsInterp - encInterp) * (valsInterp - encInterp))
    return error


dayObs = 20250708

testStart = Time("2025-07-08T20:27:00", scale='utc')
testEnd = Time("2025-07-08T21:25:00", scale='utc')
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delays_{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
goodEvents = [e for e in events if ((e.type.name=='SLEWING') and (e.duration > 10.0) and (e.begin > testStart) and (e.end < testEnd))]
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria.")

azDelays = {}
elDelays = {}
for ii in range(4): # Encoder head
    for event in goodEvents:
        seqNum = event.seqNum
        axs = []
        axs.append(fig.add_subplot(4,1,1))
        axs.append(fig.add_subplot(4,1,2))
        axs.append(fig.add_subplot(4,1,3))
        axs.append(fig.add_subplot(4,1,4))
        plt.subplots_adjust(hspace=0.5)

        az = getEfdData(
            client,
            "lsst.sal.MTMount.azimuth",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = 20.0,
            postPadding = 20.0
        )
        el = getEfdData(
            client,
            "lsst.sal.MTMount.elevation",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = 20.0,
            postPadding = 20.0
        )
        encoder = getEfdData(
            client,
            "lsst.sal.MTMount.encoder",
            event=event, 
            prePadding = 20.0,
            postPadding = 20.0
        )
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azValTimes -= azValTimes[0]
        azEncVals = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimes = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncTimes -= azEncTimes[0]

        tBegin = 5.0
        tEnd = azValTimes[-1] - 5.0
        nSteps = int((tEnd - tBegin)) * 50 + 1
        azRefTimes = np.linspace(tBegin, tEnd, nSteps)
        args = [azValues, azValTimes, azEncVals, azEncTimes, azRefTimes]
        x0 = [0.0]
        result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
        azDelay = result.x[0]
        azDelays[f"{seqNum}_{ii}"] = azDelay

        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elValTimes -= elValTimes[0]
        elEncVals = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimes = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncTimes -= elEncTimes[0]

        tBegin = 5.0
        tEnd = elValTimes[-1] - 5.0
        nSteps = int((tEnd - tBegin)) * 50 + 1
        elRefTimes = np.linspace(tBegin, tEnd, nSteps)
        args = [elValues, elValTimes, elEncVals, elEncTimes, elRefTimes]
        x0 = [0.0]
        result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
        elDelay = result.x[0]
        elDelays[f"{seqNum}_{ii}"] = elDelay


        plt.suptitle(f"Tracking event {event.seqNum} {(event.begin - TimeDelta(15.0, format='sec')).isot} - {(event.end + TimeDelta(15.0, format='sec')).isot}")                                                       
        valsInterp = np.interp(azRefTimes, azValTimes, azValues)
        encInterp = np.interp(azRefTimes, azEncTimes, azEncVals)
        axs[0].set_title("Az - no time delay")
        axs[0].plot(azRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[0].plot(azRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[0].legend()
        valsInterp = np.interp(azRefTimes, azValTimes, azValues)
        encInterp = np.interp(azRefTimes, azEncTimes - azDelay, azEncVals)
        axs[1].set_title(f"Az - {azDelay:.4f} second time delay")
        axs[1].plot(azRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[1].plot(azRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[1].legend()
        valsInterp = np.interp(elRefTimes, elValTimes, elValues)
        encInterp = np.interp(elRefTimes, elEncTimes, elEncVals)
        axs[2].set_title("El - no time delay")
        axs[2].plot(elRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[2].plot(elRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[2].legend()
        valsInterp = np.interp(elRefTimes, elValTimes, elValues)
        encInterp = np.interp(elRefTimes, elEncTimes - elDelay, elEncVals)
        axs[3].set_title(f"El - {elDelay:.4f} second time delay")
        axs[3].plot(elRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[3].plot(elRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[3].legend()
        pdf.savefig(fig)
        fig.clf()
        print(f"Finished seqNum {seqNum}, encoder head {ii}.")
pdf.close()  


In [ ]:
filename = f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delay_Dict_{dayObs}.pkl"
with open(filename, 'wb') as f:
    pkl.dump([azDelays, elDelays], f)


In [ ]:
filename = f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delay_Dict_{dayObs}.pkl"
with open(filename, 'rb') as f:
    [azDelays, elDelays] = pkl.load(f)


fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.subplots_adjust(wspace = 0.3)
plt.suptitle(f"Delay between actualPosition and encoder heads - {dayObs}")

plotRange = (0.75, 1.25)
for ii in range(4): # Encoder head
    azDels = []
    elDels = []
    for event in goodEvents:
        seqNum = event.seqNum
        key = f"{seqNum}_{ii}"
        azDels.append(azDelays[key])
        elDels.append(elDelays[key])
    axs[0].hist(azDels, bins=50, alpha=0.5, range = plotRange, edgecolor='black', label=f"Head{ii}")
    axs[1].hist(elDels, bins=50, alpha=0.5, range = plotRange, edgecolor='black', label=f"Head{ii}")
axs[0].set_title("Azimuth delays")
axs[0].set_xlim(0.8, 1.2)
axs[0].set_xlabel("Delay(seconds)")
axs[0].legend()
axs[1].set_title("Elevation delays")
axs[1].set_xlim(0.8, 1.2)
axs[1].set_xlabel("Delay(seconds)")
axs[1].legend()
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delay_Summary_{dayObs}.png")